### This notebook has all the functions for the Lorenz63 model.



In [ ]:
# Calling libraries:
from __future__ import division
%matplotlib inline
import numpy as np
import time
from scipy.stats import norm, uniform, bernoulli
from scipy.linalg import sqrtm 
import math 
from resampling_functions import resample, systematic_resampling
import scipy
import multiprocessing as mp
import parmap

* Here $W$ is $N$ iid copies of $N(0,1)$.

In [ ]:
def propagate_Lorenz63(X, W, dt, delta, theta) :
    """
    usual: 3D Lorenz model + multiplicative noise
    compute one step forward, usual Euler-Maruyama discretization
    """
    sigma = theta[0]
    rho   = theta[1]
    beta  = theta[2]
    noise_intensity = theta[3]
    #ODE forward + noise
    N = np.shape(X)[1]
    sqdt = np.sqrt(dt)

    for i in range( int(delta/dt) ) :
        xx = np.zeros(( 3, N ))
        xx[0,:] = X[0,:] + dt*sigma*(X[1,:] - X[0,:]) + noise_intensity*X[0,:]*sqdt*W[i,:]
        xx[1,:] = X[1,:] + dt*(X[0,:]*(rho - X[2,:]) - X[1,:]) + noise_intensity*X[1,:]*sqdt*W[i,:]
        xx[2,:] = X[2,:] + dt*(X[0,:]*X[1,:] - beta*X[2,:]) + noise_intensity*X[2,:]*sqdt*W[i,:]
        X [:,:] = xx[:,:]
    
    return X

#### Simulate data:

* Define a function to simulate observations.

* $\theta = (\sigma, \rho, \beta, \sigma_{\text{latent}})$.

In [ ]:
def simulate_observations(x_0, T, dt, delta, H, R, theta) : 
    """
    generate a sequence of observations x_0 : initialization
    T = number of observations
    delta = time between observations
    """
    m  = np.shape(H)[0]
    y  = np.zeros(( m, T ))
    X  = np.zeros(( 3, 1 ))
    trajectory = np.zeros(( 3, T ))
    X[:,0] = x_0
    
    for t in range(T) :
        W = np.random.randn(int(delta/dt),1)
        X = propagate_Lorenz63(X, W, dt, delta, theta)
        trajectory[:,t] = X[:,0]
        y[:,t] = np.dot(H, X[:,0]) + np.random.multivariate_normal(np.zeros(m), R, 1)
                        
    return y, trajectory

* First define $g$, the error functiom (that is, the density of $Y_t$ given $X_t$).